<a href="https://colab.research.google.com/github/uzaramen108/fenicsx-colab/blob/main/tutorial/fenics_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [1]:
# --------------------------------------------------
# 1️⃣ Mount Google Drive (optional, for cache)
# --------------------------------------------------
from google.colab import drive
import os

if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")
else:
    print("📦 Google Drive already mounted")

# --------------------------------------------------
# 2️⃣ Clone fenicsx-colab repository (idempotent)
# --------------------------------------------------
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    subprocess.run(cmd, check=True)

if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repository already exists — skipping clone")

# --------------------------------------------------
# 3️⃣ Run setup_fenicsx.py IN THIS KERNEL (CRITICAL)
# --------------------------------------------------
print("🚀 Running setup_fenicsx.py in current kernel")

# ⚙️ Configuration
USE_COMPLEX = True  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = True    # <--- Set True to remove existing environment

# Build options
opts = []
if USE_COMPLEX:
    opts.append("--complex")
if USE_CLEAN:
    opts.append("--clean")

opts_str = " ".join(opts) if opts else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}"
)

# --------------------------------------------------
# 4️⃣ Sanity check
# --------------------------------------------------
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

Mounted at /content/drive
📥 Cloning fenicsx-colab...
🚀 Running setup_fenicsx.py in current kernel
🔧 FEniCSx Setup Configuration
PETSc type      : complex
Clean install   : True

📦 Google Drive detected — using persistent cache

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Complex PETSc (complex128)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Complex PETSc is installed
   - Use for eigenvalue problems, frequency-domain analysis
   - Some examples may require real PETSc

🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI 

# 실행 파일

In [30]:
%%fenicsx -np 4
from mpi4py import MPI
from dolfinx import mesh
from dolfinx import fem
from dolfinx import default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import numpy
import ufl

domain = mesh.create_unit_square(MPI.COMM_WORLD, 8, 8, mesh.CellType.quadrilateral)
V = fem.functionspace(domain, ("Lagrange", 1))

uD = fem.Function(V)
uD.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)

tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)

boundary_dofs = fem.locate_dofs_topological(V, fdim, boundary_facets)
bc = fem.dirichletbc(uD, boundary_dofs)


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

f = fem.Constant(domain, default_scalar_type(-6))

a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = f * v * ufl.dx

problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
    petsc_options_prefix="poisson"
)
uh = problem.solve()

V2 = fem.functionspace(domain, ("Lagrange", 2))
uex = fem.Function(V2, name="u_exact")
uex.interpolate(lambda x: 1 + x[0] ** 2 + 2 * x[1] ** 2)

L2_error = fem.form(ufl.inner(uh - uex, uh - uex) * ufl.dx)
error_local = fem.assemble_scalar(L2_error)
error_L2 = numpy.sqrt(domain.comm.allreduce(error_local, op=MPI.SUM))

error_max = numpy.max(numpy.abs(uD.x.array - uh.x.array))
if domain.comm.rank == 0:
    print(f"Error_L2 : {error_L2:.2e}")
    print(f"Error_max : {error_max:.2e}")

# XDMF/HDF5 파일로 저장 (ParaView 사용)
from dolfinx import io
from pathlib import Path

results_folder = Path("results")
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / "fundamentals"

# XDMF 파일로 저장
with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh)

if domain.comm.rank == 0:
    print(f"\n✅ Files saved:")
    print(f"   - {filename.with_suffix('.xdmf')}")
    print(f"   - {filename.with_suffix('.h5')}")
    print(f"\n📊 To visualize in ParaView:")
    print(f"   1. Open ParaView")
    print(f"   2. File → Open → {filename.with_suffix('.xdmf')}")
    print(f"   3. Click 'Apply' to load the data")
    print(f"   4. Select 'u' variable to visualize the solution")

Error_L2 : 8.24e-03
Error_max : 1.55e-15

✅ Files saved:
   - results/fundamentals.xdmf
   - results/fundamentals.h5

📊 To visualize in ParaView:
   1. Open ParaView
   2. File → Open → results/fundamentals.xdmf
   3. Click 'Apply' to load the data
   4. Select 'u' variable to visualize the solution


In [13]:
%%fenicsx -np 4
from mpi4py import MPI
from dolfinx import mesh
from dolfinx import fem
from dolfinx import default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import numpy as np
import ufl
from pathlib import Path

# Create mesh and function space
domain = mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)
V = fem.functionspace(domain, ("Lagrange", 1))

# Create test functions
u_r = fem.Function(V, dtype=np.float64)
u_r.interpolate(lambda x: x[0])
u_c = fem.Function(V, dtype=np.complex128)
u_c.interpolate(lambda x: 0.5 * x[0] ** 2 + 1j * x[1] ** 2)

tdim = domain.topology.dim
fdim = tdim - 1

print("u_r dtype:", u_r.x.array.dtype)
print("u_c dtype:", u_c.x.array.dtype)

# PETSc setup
from petsc4py import PETSc
from dolfinx.fem.petsc import assemble_vector

print("PETSc.ScalarType:", PETSc.ScalarType)
assert np.dtype(PETSc.ScalarType).kind == "c", "PETSc must be compiled with complex support"

# Variational formulation
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(-1 - 2j))

a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = ufl.inner(f, v) * ufl.dx
L2 = f * ufl.conj(v) * ufl.dx

print("L  =", L)
print("L2 =", L2)

# Test derivative
J = u_c**2 * ufl.dx
F = ufl.derivative(J, u_c, ufl.conj(v))
residual = assemble_vector(fem.form(F))
print("Residual:", residual.array)

# Boundary conditions
domain.topology.create_connectivity(fdim, tdim)
boundary_facets = mesh.exterior_facet_indices(domain.topology)
boundary_dofs = fem.locate_dofs_topological(
    V, fdim, boundary_facets
)
bc = fem.dirichletbc(u_c, boundary_dofs)

# Solve the problem
problem = fem.petsc.LinearProblem(
    a, L, bcs=[bc], petsc_options_prefix="complex_poisson"
)
uh = problem.solve()

# Compute error
x = ufl.SpatialCoordinate(domain)
u_ex = 0.5 * x[0] ** 2 + 1j * x[1] ** 2
L2_error = fem.form(
    ufl.dot(uh - u_ex, uh - u_ex) * ufl.dx(metadata={"quadrature_degree": 5})
)
local_error = fem.assemble_scalar(L2_error)
global_error = np.sqrt(domain.comm.allreduce(local_error, op=MPI.SUM))
max_error = domain.comm.allreduce(np.max(np.abs(u_c.x.array - uh.x.array)), op=MPI.MAX)

print(f"L2 Error: {global_error}")
print(f"Max Error: {max_error}")

# --------------------------------------------------
# Export to HDF5/XDMF for Paraview
# --------------------------------------------------
from dolfinx.io import XDMFFile

# Create output directory
output_dir = Path("complex_poisson_output")
output_dir.mkdir(exist_ok=True)

# We need to split complex function into real and imaginary parts
# because Paraview doesn't natively support complex fields

V_real = fem.functionspace(domain, ("Lagrange", 1))
uh_real = fem.Function(V_real, name="uh_real")
uh_imag = fem.Function(V_real, name="uh_imag")

# Copy real and imaginary parts
uh_real.x.array[:] = uh.x.array.real
uh_imag.x.array[:] = uh.x.array.imag

# Export real part
with XDMFFile(domain.comm, output_dir / "uh_real.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh_real)

# Export imaginary part
with XDMFFile(domain.comm, output_dir / "uh_imag.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(uh_imag)

if domain.comm.rank == 0:
    print(f"\n✅ XDMF files exported to {output_dir}/")
    print(f"   - uh_real.xdmf")
    print(f"   - uh_imag.xdmf")
    print(f"\nOpen these files in Paraview to visualize the solution.")

u_r dtype: float64
u_c dtype: complex128
PETSc.ScalarType: <class 'numpy.complex128'>
L  = { c_0 * (conj((v_0))) } * dx(<Mesh #0>[everywhere], {})
L2 = { c_0 * (conj((v_0))) } * dx(<Mesh #0>[everywhere], {})
Residual: [1.66666667e-05+0.00843333j 4.16666667e-06+0.003175j
 5.00000000e-05+0.00936667j 1.33333333e-04+0.01626667j
 1.66666667e-05+0.0067j     1.83333333e-04+0.00936667j
 1.33333333e-04+0.01286667j 4.33333333e-04+0.01626667j
 1.66666667e-05+0.00516667j 4.16666667e-04+0.00936667j
 1.33333333e-04+0.00986667j 4.33333333e-04+0.01286667j
 9.33333333e-04+0.01626667j 1.66666667e-05+0.00383333j
 7.50000000e-04+0.00936667j 1.33333333e-04+0.00726667j
 4.33333333e-04+0.00986667j 9.33333333e-04+0.01286667j
 1.63333333e-03+0.01626667j 1.66666667e-05+0.0027j
 1.33333333e-04+0.00506667j 4.33333333e-04+0.00726667j
 9.33333333e-04+0.00986667j 1.66666667e-05+0.00176667j
 1.33333333e-04+0.00326667j 4.33333333e-04+0.00506667j
 9.33333333e-04+0.00726667j 1.25000000e-05+0.000775j
 4.16666667e-04+0.00